# <span id="chap-parallel-async"></span> Working asynchronously

As will now be abundantly clear, performing research-grade simulation is very time-consuming even if you have access to research-grade computing power &ndash; and many of us don't, at least a lot of the time. In any event, running a large simulation from IPython will take a long time. Using the techniques we've used above, we're locked-out of the notebook while the simulation is on-going. We can't carry on writing in the notebook, or doing other useful work. Worse, we can't disconnect the notebook from the cluster for the duration of the simulation, as doing so will lose the data we've spent so much effort computing. This is bad for workstation users &ndash; the machine can't be re-booted, a crash will lose data, and so on &ndash; but it's *really* bad for laptop users, who can't head off on the road. And let's face it, most scientists these days use laptops as personal machines: indeed, one of the great things about compute clusters is that you don't *need* a big, dedicated machine on your desk.

Fortunately, there is an alternative. It involves starting computing jobs *asynchronously*, without waiting for them to complete as happens normally. We can then disconnect from the cluster while it gets on with doing our simulations, and then re-connect later to pick up the results. Doing this requires a small change in the set-up of the cluster we're using, and a small change in the way we code the client-side of our computation, but no changes at all in the simulation part.

## <span id="sec_parallel_async_asynchronous"></span> Asynchronous computing

So what is asynchronous computing?

We can take a step back and note that the computing we've done so far has been *synchronous*. What this means is that we request that a computation takes place and wait to get its results, before going on to the next computation. The fact that a computation is happening ona  remote cluster, using a parellel map operation, doesn't matter: we set the calculation going and wait for it to complete. If it takes a long time, we wait a long time, during which we're locked-out of doing other work.

*Asynchronous* computing is therefore computing that isn't synchronous. Instead of waiting while a computation takes place, we instead get the opportunity to continue with other work and then "come back later" to pick up the results. Most of the time such an approach would be a nightmare: if the second thing we want to do depends on the first having been done, then it's far better to just wait while the first thing completes. But where this *isn't* the case, where we don't need the results immediately in order to do other things, then it can be advantageous to not *have* to wait. This is especially true when a computation is going to take hours (or days).

## <span id="sec_parallel_async_cluster"></span> Setting up a cluster for asynchronous computing

So in IPython terms, asynchronous operation lets us submit jobs to a controller and then disconnect, returning later to collect the results. This implies that the controller we connect to remembers the results of calculations until we come back for them. Moreover, re-connection means that, from the controller's perspective, a *new* client connection is asking for the results of a *different* client connection's computations.

The default cluster computing profile takes a simple approach. It assumes that a client asks for its own results, and forgets uncollected results when a client disconnects. This is actually a pretty good choice of default behaviour, since it allows things like crashed notebooks to be handled without tying-up the cluster. It means however that we need to re-configure our cluster slightly to support asynchronous use: to remember results and allow re-connection. As a side benefit, this also makes the controller more robust to engine failures.

We [discussed earlier](parallel-smple.ipynb#sec_parallel_simple_cluster_mechanics) how each IPython compute job is bound to a message with a message identifier, with the controller associating the results of each calculation with the appropriate message ids as the calculations complete. By default the controller stores these associations in memory: the tweak we'll make is to have it store the message ids and their results in a database that persists over time. This means that a client can request the result of a specific message and retrieve it from the database (if it's been computed already), wait for it (if it hasn't), or query whether it's ready or not and come back later.

IPython can use a number of different databases to manage its message store. The simplest is to use SQLite, support for which is built-in to Python. (You can also use MongoDB or other database providers for special circumstances, but the purposes of this book we'll stick to SQLite as it's proved robust and scalable enough.)

Making use of SQLite as message store involves changing a single configuration parameter. We again edit `.ipython/profile_sample/ipcontroller_config.py` (assuming again that our profile is called *sample*) and add another configuration line:

<pre>
c.HubFactory.db_class ="IPython.parallel.controller.sqlitedb.SQLiteDB"
</pre>

Starting the cluster will then create a database called `/ipython/profile_profile/tasks.db` that will be populated with incoming messages and their computed results. (Other configuration options let you rename or re-locate this database, again for special circumstances that we won't go into.)

## <span id="sec_parallel_async_jobs"></span> Submitting jobs asynchronously

TBC

## <span id="sec_parallel_async_checking"></span> Checking for completion

Now we have a set of compute jobs executing on our cluster, and a set of message corresponding to those jobs. We need to be able to check whether the jobs have finished, and then collect the results. Both these tasks can be performed easily from inside an IPython notebook, without blocking.

Suppose our computations have been running for some time, and we want to check on them. The following code will accomplish this:

In [ ]:
# re-connect back to the cluster
client2 = Client(profile = 'sample')

# re-load the message identifiers
with open('jobs.pickle', 'rb') as handle:
    jobs = pickle.load(handle)
    
# query job state
status = [ client2.get_result(msg).ready() for msg in jobs ]
print '{n} jobs running, {c}% complete'.format(n = len(status),
                                               c = int(((len(filter(lambda f: f, status)) + 0.0) / len(status)) * 100))

# tidy up
client2.close()

We first re-connect to the cluster and re-load the message ids of our jobs. We then use `get_result()` to present the message ids one by one to the cluster. `get_result()` returns an `AsyncResult` object, a Python object that represents the current state of an asynchronously-executed job. Calling `ready()` on this object tests whether the job's result is ready or not, so we end up with a list of ready values, `True` for jobs that have completed and `False` for those that are still on-going. We then present these as a percentage complete. (Closing the connection to the cluster is important, as otherwise the underlying Python engine can run out of file handles if you keep a notebook open for a long time and check often.)

We can use this code to query that state of our computations. If they're not complete, go away and wait again. Eventually, all being well, all the results will be ready and we can retrieve them.

## <span id="sec_parallel_async_retrieving"></span> Retrieving results

Once everything is ready, we can use the `get()` method on `AsyncResult` to retrieve the results:

In [ ]:
# retrieve the results
client2 = Client(profile = 'sample')
results = client2.get_result(jobs).get()
client2.close()

This leaves the results of all the jobs associated with the message ids in a single list in `results`. `get()` is a synchronous method: it blocks until the results are available. We could call it anytime if we're willing to go synchronous, but in this case we already know that all the results are available, so we know that `get()` will successfully retrieve the results without waiting.

It's important to note three things at this point. Firstly, we can disconnect the client completely from the cluster. Since some simulations might take a week (or more), this is clearly a useful thing to be able to do, especially if you're a laptop-user. Secondly, to get this functionality, we've changed the set-up of the cluster and added some client code, but we haven't changed the simulation code at all: asynchronous operation in IPython is purely about how you *call* code, not about the *called* code at all. This isn't true of other computing frameworks, and is quite a benefit: it means that IPython simulations scale-out much more easily, since you can run them small and debug them locally and synchronously before running them asynchronously for the real task. Finally, the optimisations we identified to avoid moving data over the network, and to maximise parallelism, all work just as well with asynchronous operation as with synchronous, so moving to asynchronous coding just adds some more flexibility, again without having to re-think the simulation code too much.  